In [ ]:
using QuantumCollocation
using NamedTrajectories
using TrajectoryIndexingUtils

using Interpolations
using LinearAlgebra
using SparseArrays

# Plots
using CairoMakie
using Colors
using Printf

In [ ]:
const Units = 1e9
const MHz = 1e6 / Units
const GHz = 1e9 / Units
const ns = 1e-9 * Units
const μs = 1e-6 * Units
;

In [ ]:
# Operators
const Paulis = Dict(
    "I" => Matrix{ComplexF64}(I, 2, 2),
    "X" => Matrix{ComplexF64}([0 1; 1 0]),
    "Y" => Matrix{ComplexF64}([0 -im; im 0]),
    "Z" => Matrix{ComplexF64}([1 0; 0 -1]),
)
const a = [0 1; 0 0]
const ad = transpose(a);
excitation(theta) = exp(-im/2 * theta * Paulis["Z"]);

H_drift = [ ]

H_drives = [
     Paulis["X"],
     Paulis["Y"]
]
system = QuantumSystem(H_drives);
t_f = 20 * ns
n_steps = 101
times = range(0, t_f, n_steps)  # Alternative: collect(0:Δt:t_f)
Δt = times[2] - times[1]

In [ ]:
### Generate Initial Trajectories 
PICO_max_iter = 100

# Shape the cost function with weights on states and controls
Q = 1000.
R =1e-2
# Add control bounds
a_bound = 2 * π * 200 * MHz
dda_bound = 1e-2

initial_infidelities = []
initial_trajectories = []
initial_problems = []

ops = Options()
ops.print_info_string = "yes"
ops.recalc_y = "yes"
ops.recalc_y_feas_tol = 1.0 ##down
ops.print_level = 2

N = 11

for theta in range(0,2*pi, N )    
    
    print("-----------------------")
    print("Angle "*string(theta))
    println("-----------------------")

    target = excitation(theta)    
    p = UnitarySmoothPulseProblem(
        system,
        target,
        n_steps,
        Δt;
        a_bound=a_bound,
        dda_bound=dda_bound,
        Q=Q,
        R=R,
        R_dda=R*10,
        verbose=true,
        hessian_approximation=true,
        pade_order=10,
        free_time=false,
        timesteps_all_equal=true,
      max_iter=PICO_max_iter,
        ipopt_options=ops,
        #geodesic=true
    )
    push!(initial_problems,p)
    
end



In [ ]:
trajectories = []
for n in range(1,N-1)
    problems = [initial_problems[n],initial_problems[n+1]]
    print("-----------------------")
    print("Angle "*string(range(0,2*pi, N )[n]))
    println("-----------------------")
    
        p = UnitaryDirectSumProblem(
                problems, 
                1-1e-5;
                Q=Q,
                R=R,
                verbose=true,
                hessian_approximation=true,
                pade_order=10,
                autodiff=false,
                free_time=false,
                timesteps_all_equal=true,
                max_iter=PICO_max_iter,
                ipopt_options=ops,
                #geodesic=true
            )
            solve!(p)  
    push!(trajectories,p.trajectory[:a])
end


In [ ]:
using Interpolations

DATA=[]

for theta in range(0,2*pi,convert(Int64,2500))
    control = nothing 
    if(theta in range(0, 2*pi, N ))
        i = findfirst(item -> item == theta, range(0,2*pi, N ))
        #control = trajectories[index][[1,2,3,4,5],:]
        if(i<N)
            #control = trajectories[i][[1,2,3,4,5],:]
            control = trajectories[i][[1,2],:]
        else
            #control = trajectories[N-1][[6,7,8,9,10],:]
            control = trajectories[N-1][[3,4],:]
        end
    else
        i  = ceil(Int, (N-1)/(2*pi) * theta )
        #control_interp = Interpolations.linear_interpolation([range(pi/N,pi*(2*N -1)/N,N)[i],range(pi/N,pi*(2*N -1)/N,N)[i+1]],[trajectories[i][[1,2,3,4,5],:],trajectories[i][[6,7,8,9,10],:]]  )
        #control_interp = Interpolations.linear_interpolation([range(0,2*pi,N)[i],range(0,2*pi,N)[i+1]],[trajectories[i][[1,2],:],trajectories[i][[3,4],:]]  )
        #control_interp = Interpolations.linear_interpolation([range(0,2*pi,N)[i],range(0,2*pi,N)[i+1]],[trajectories[i][[1,2,3,4,5],:],trajectories[i][[6,7,8,9,10],:]]  )
        control_interp = Interpolations.linear_interpolation([range(0,2*pi,N)[i],range(0,2*pi,N)[i+1]],[trajectories[i][[1,2],:],trajectories[i][[3,4],:]]  )
        control = control_interp(theta)
    end
    #x=operator_to_iso_vec([1 0 0 0;0 1 0 0; 0 0 1 0; 0 0 0 1])
    x=operator_to_iso_vec([1 0 ;  0 1])
    rollout_states = unitary_rollout(x,control,Δt,system)
    push!(DATA,(unitary_infidelity(rollout_states[:, end], operator_to_iso_vec(excitation(theta)))))
end

In [ ]:

f = Figure()
ax = Axis(f[1, 1],
    title = "Infidelity Log Plot",
    xlabel = "θ",
    ylabel = "Infidelity"
)
lines!(ax, range(0,2*pi,convert(Int64,2500)), log10.(convert(Array{Float64,1},DATA)), label  = "Linear",color = :blue)


f[1, 2] = Legend(f, ax, "Infidelity Data", framevisible = false)
f
     

In [ ]:
d=[]
for i in range(1,N-1)
    push!(d,trajectories[i][[1,2],:])
end
push!(d,trajectories[N-1][[3,4],:])

In [ ]:
f = Figure()
ax1 = Axis(f[1, 1])
lines!(ax1, d[2][1,:], color=:lightblue, linewidth=5)
lines!(ax1, d[2][2,:], color=:lightblue, linewidth=5)

f

In [ ]:
initial_x_train = []
for m in range(1,n_steps)
    for n in range(1,N)
        push!(initial_x_train,range(0,2*pi, N )[n])
        push!(initial_x_train,m)
    end
end
initial_x_train = convert(Matrix{Float32},reshape(initial_x_train,(2,N*n_steps)))

In [ ]:
initial_y_train=[]
for m in range(1,n_steps)
    for n in range(1,N)
        push!(initial_y_train,d[n][:,m][1])
        push!(initial_y_train,d[n][:,m][2])
    end
end
initial_y_train = convert(Matrix{Float32},reshape(initial_y_train,(2,N*n_steps)))

In [ ]:
using Flux

In [ ]:
layer1 = Dense(2 => 20,relu)
layer2 = Dense(20 => 20,relu)
layer3 = Dense(20 => 20,relu)
layer4 = Dense(20 => 20,relu)
layer5 = Dense(20 => 20,relu)
layer6 = Dense(20 => 20,relu)
layer7 = Dense(20 => 20,relu)
layer8 = Dense(20 => 20,relu)
layer9 = Dense(20 => 20,relu)
layer10 = Dense(20 => 20,relu)
layer11 = Dense(20 => 20,relu)
layer12 = Dense(20 => 20,relu)
layer13 = Dense(20 => 20,relu)
layer14 = Dense(20 => 20,relu)
layer15 = Dense(20 => 2)
predict = layer15 ∘layer14 ∘layer13 ∘layer12 ∘layer11 ∘ layer10 ∘ layer9 ∘ layer8 ∘ layer7 ∘ layer6 ∘ layer5 ∘ layer4∘ layer3∘ layer2 ∘ layer1


In [ ]:
predict(initial_x_train)

In [ ]:
using Statistics
loss(model, x, y) = mean(abs2.(model(x) .- y));
loss(predict,initial_x_train,initial_y_train)

In [ ]:
using Flux: train!
opt = Flux.setup(ADAM(), predict)
data = [(initial_x_train, initial_y_train)]

In [ ]:
losses = []
for epoch in 1:1000000
         train!(loss, predict, data, opt)
    if(epoch % 1000 == 0)
        l=loss(predict,initial_x_train,initial_y_train)
        @printf("Epoch: %d  Loss: %.4f \n",epoch,l)
        push!(losses,l)
    end
end

In [ ]:
loss(predict, initial_x_train, initial_y_train)


In [ ]:
predicted_data =[convert(Matrix{Float64},reduce(hcat,[predict([theta,i]) for i in range(1,n_steps)])) for theta in range(0,2*pi,N)];

In [ ]:
f = Figure()
ax1 = Axis(f[1, 1])
lines!(ax1, d[5][1,:], color=:lightblue, linewidth=5)
lines!(ax1, d[5][2,:], color=:lightblue, linewidth=5)

lines!(ax1, predicted_data[5][1,:], color=:blue, linewidth=5)
lines!(ax1, predicted_data[5][2,:], color=:blue, linewidth=5)

f

In [ ]:
f = Figure()
ax1 = Axis(f[1, 1])
lines!(ax1, convert(Vector{Float32},losses), color=:lightblue, linewidth=5)


f

In [ ]:
x=[]
y=[]
for col in eachcol(initial_x_train)
    push!(x,col[1])
    push!(y,col[2])
end

z1=[]
z2=[]
for col in eachcol(initial_y_train)
    push!(z1,col[1])
    push!(z2,col[2])
end

NN_z1=[]
NN_z2=[]
for i in range(1,length(x))
    p=predict([x[i],y[i]])
    push!(NN_z1,p[1])
    push!(NN_z2,p[2])
end



In [ ]:
surface(convert(Vector{Float32},x/(2*pi)), convert(Vector{Float32},y/n_steps), convert(Vector{Float32},z1))


In [ ]:
surface(convert(Vector{Float32},x/(2*pi)), convert(Vector{Float32},y/n_steps), convert(Vector{Float32},NN_z1))

In [ ]:
get_control(model,theta) = convert(Matrix{Float64},reduce(hcat,[model([theta,i]) for i in range(1,n_steps)]))

function new_rollout(controls::AbstractMatrix{Float64})

    
    T = size(controls, 2)
    ts = fill(Δt, T)

    Ũ⃗ = [1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 1 ]

    G_drift = Matrix{Float64}(system.G_drift)
    G_drives = Matrix{Float64}.(system.G_drives)
    for t = 2:T
        aₜ₋₁ = controls[:, t - 1]
        Gₜ = Integrators.G(
            aₜ₋₁,
            G_drift,
            G_drives
        )
         Ũ⃗= exp(Gₜ * ts[t - 1]) *  Ũ⃗
    end

    return Ũ⃗[[1,2],[1,2]]+im*Ũ⃗[[3,4],[1,2]]
end
using Statistics
infid(model,theta)=1-abs(tr(new_rollout(get_control(model,theta))'excitation(theta)))/2


In [ ]:
infde =[]

for theta in range(0,2*pi,1000)
    push!(infde,infid(predict,theta))
end

In [ ]:
plot(log10.(convert(Vector{Float32},(infde))))

In [ ]:
loss(model,x,y)=mean(infid.(model,x))
x_train = Vector{Float32}(range(0,2*pi,100))
y_train = range(0,2*pi,100)
loss(predict,x_train,y_train)

In [ ]:
using Flux: train!
opt = Flux.setup(ADAM(), predict)
data = [(x_train, y_train)]

In [ ]:
losses = []
for epoch in 1:10000
        train!(loss, predict, data, opt)
    if(epoch%100==0)
        l=loss(predict,initial_x_train,initial_y_train)
        @printf("Epoch: %d  Loss: %.4f \n",epoch,l)
        push!(losses,l)
    end
end

In [ ]:
loss(predict,x_train,y_train)

In [ ]:
plot(convert(Vector{Float64},losses))

In [ ]:
infde =[]

for theta in range(0,2*pi,1000)
    push!(infde,infid(predict,theta))
end

In [ ]:
plot(log10.(convert(Vector{Float32},(infde1))))